In [2]:
!pip install selenium
!pip install bs4
# brew install chromedriver via terminal before continue

In [117]:
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from bs4 import BeautifulSoup 
from selenium.webdriver.common.by import By

chrome_options = webdriver.ChromeOptions()
chrome_options.add_argument('--no-sandbox')
driver = webdriver.Chrome()
driver.implicitly_wait(30)
driver.maximize_window()




In [121]:
def login():
    try:
        driver.get('https://www.finnomena.com/')
        driver.find_element_by_link_text('ลงชื่อเข้าใช้').click()
        driver.find_element_by_id('username-login').send_keys('')
        driver.find_element_by_id('username-login').send_keys('###EMAIL###')
        driver.find_element_by_class_name('next-process').click()
        driver.find_element_by_id('password-login').send_keys('###PASSWORD###')
        driver.find_element_by_class_name('submit-btn').click()
    except Exception as e:
        print(e)
login()

In [153]:
def getStockDetails(name,isYearly):
    # stock-growthrate
    sections = ['stock-growthrate','stock-financial-report','stock-financial-ratio','stock-statistics']
    url = 'https://www.finnomena.com/stock/' + name
    driver.get(url)
    headers = []
    quarters = []
    data = []
    toggled = False
    try:
        for section in sections:
            obj = {}
            keys = []
            headerElements = driver.find_element_by_css_selector(f'#{section}>div>div>div>div>div.table-overflow-wrapper>div.topic-wrapper.user-select-none.float-left.overflow-shadow')
            if len(headers) == 0:
                for div in BeautifulSoup(headerElements.get_attribute('innerHTML'), 'html.parser').find_all('div', {'class': 'topic'}):
                    headers.append(div.get_text())
            if not toggled and isYearly:
                toggled = True
                driver.find_element_by_class_name('toggle').click()
                
            for topic in headerElements.find_elements_by_class_name('topic')[1:]:
                keys.append(BeautifulSoup(topic.get_attribute('id')).get_text())
            contentElements = driver.find_element_by_css_selector(f'#{section}>div>div>div>div>div.table-overflow-wrapper>div.content-wrapper.user-select-none')
            contentHTML = BeautifulSoup(contentElements.get_attribute('innerHTML'), 'html.parser')
            dataWrapper = contentHTML.find_all('div', {'class': 'data-wrapper'})
            
            if len(quarters) == 0:
                for div in contentHTML.find_all('div', {'class': 'year'}):
                    quarters.append(div.get_text())
            
            for i in range(len(dataWrapper)): 
                values = [data.get_text() for data in dataWrapper[i].find_all('div', {'class': 'data-each'})]
                obj[keys[i]] = values
            
            data.append(obj)
        return (headers,quarters,data)
    except Exception as e:
        print(e) 

In [154]:
import pandas as pd
headers,quartersOrYear,responseData = getStockDetails(name='mint',isYearly=True)


In [163]:
data = {}
for d in responseData:
    data = dict(list(d.items()) + list(data.items()))
data['index'] = quartersOrYear
df = pd.DataFrame(data) 
df = df.iloc[:-1]
df.set_index('index',inplace = True)
df.transpose()

index,2553,2554,2555,2556,2557,2558,2559,2560,2561,2562
Close,11.80,11.20,19.60,20.70,32.50,36.25,35.75,43.75,34.00,36.00
MKTCap,"38,495.60","36,682.52","72,260.63","82,828.06","130,050.59","159,583.80","157,670.67","202,077.50","157,043.09","166,284.16"
PriceEarningRatio,27.01,12.91,26.80,22.21,29.97,30.46,18.09,39.64,28.27,19.73
PriceBookValue,3.18,2.63,4.35,3.57,4.92,5.41,4.34,5.20,2.58,2.29
BookValuePerShare,3.71,4.26,4.54,5.81,6.61,6.70,8.23,8.79,13.19,15.73
DividendYield,1.27,1.34,0.68,1.43,1.08,0.63,0.98,0.76,1.18,1.11
EVPerEbitDA,15.34,9.84,14.07,13.42,18.94,17.99,16.58,21.56,20.80,12.10
ROA,4.07,7.82,7.42,7.40,6.56,8.28,6.45,4.77,2.82,4.10
ROE,10.31,21.11,20.98,18.98,16.56,23.14,18.77,12.78,9.13,14.52
GPM,54.67,52.48,53.10,54.47,54.46,51.10,51.59,55.17,53.16,41.44
